Task 1:

In [4]:
!curl http://localhost:9200

{
  "name" : "c7d84133cab9",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "d1MxJZHeRKeAuY9qNPed_Q",
  "version" : {
    "number" : "9.0.0",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "112859b85d50de2a7e63f73c8fc70b99eea24291",
    "build_date" : "2025-04-08T15:13:46.049795831Z",
    "build_snapshot" : false,
    "lucene_version" : "10.1.0",
    "minimum_wire_compatibility_version" : "8.18.0",
    "minimum_index_compatibility_version" : "8.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   540  100   540    0     0  21724      0 --:--:-- --:--:-- --:--:-- 22500


Import dependencies

In [5]:
from tqdm.auto import tqdm
import requests
from elasticsearch import Elasticsearch
import tiktoken
from openai import OpenAI
import os

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download documents

In [6]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Task 2:

In [7]:
es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:52<00:00, 18.02it/s]


Task 3:

In [ ]:
#No filters are applied to the search results
prompt1 = "How do execute a command on a Kubernetes pod?"

search_prompt1 = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": prompt1,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            }
        }
    }
}

result_1 = es_client.search(index=index_name, body=search_prompt1)
result_1['hits']['hits'][0]['_score']


44.50556

Task 4:

In [ ]:
#A filter is applied to the search results. Only documents where the course field is equal to machine-learning-zoomcamp will be selected in the results.
search_prompt2 = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": prompt1,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

result2 = es_client.search(index=index_name, body=search_prompt2)
print(result2)

result2['hits']['hits'][2]['_source']["question"]

{'took': 57, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 303, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'POdedpcBL0SOirgykZ8W', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'y-dedpcBL0SOirgyrp_V', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard', '

'How do I copy files from a different folder into docker container’s working directory?'

Task 5:

In [ ]:
prompt2="How do copy a file to a Docker container?"

#A filter is applied to the search results. Only documents where the course field is equal to machine-learning-zoomcamp will be selected in the results.
search_prompt3 = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": prompt2,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

result3 = es_client.search(index=index_name, body=search_prompt3)

In [16]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for doc_json in result3['hits']['hits']:
    doc = doc_json['_source']
    context += context_template.format(question=doc["question"], text=doc["text"]).strip() + "\n\n"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

final_prompt = prompt_template.format(question=prompt2, context=context).strip()
print(final_prompt)
len(final_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

1446

Task 6:

In [17]:
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(final_prompt))

320

Bonus task:

In [ ]:
API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=API_KEY)
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[{"role": "user", "content": final_prompt}]
)

response=completion.choices[0].message.content
print(response)


You can copy a file from your local machine to a Docker container using the `docker cp` command. The basic syntax for copying a file into a running Docker container is:

```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```


In [ ]:
#Theoretical price in dollars ($)
input_tokens = 150
output_tokens = 250
requests = 1000
input_cost = 0.005
output_cost = 0.015

price = (input_tokens * input_cost + output_tokens * output_cost) * requests / 1000
print(price)

4.5


In [ ]:
#Real price in dollars ($)
input_tokens = len(encoding.encode(final_prompt))
output_tokens = len(encoding.encode(response))

price = (input_tokens * input_cost + output_tokens * output_cost) * requests / 1000
print(price)

2.41
